In [ ]:
!pip install nipype

In [1]:
# import the relevant Python packages
import numpy
import nibabel
import nipype
import matplotlib

In [2]:

!pip install gdown


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! pip install jedi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.9 MB/s 


In [5]:
!pip install pypet2bids

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## download file and upload the zip file to the Files:
[https://drive.google.com/file/d/10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD/view?usp=sharing](https://drive.google.com/file/d/10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD/view?usp=sharing)

In [10]:
!unzip OpenNeuroPET-Demo_raw.zip

Archive:  OpenNeuroPET-Demo_raw.zip
   creating: OpenNeuroPET-Demo_raw/
  inflating: __MACOSX/._OpenNeuroPET-Demo_raw  
  inflating: OpenNeuroPET-Demo_raw/.DS_Store  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._.DS_Store  
   creating: OpenNeuroPET-Demo_raw/sub-SiemensHRRT/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._sub-SiemensHRRT  
   creating: OpenNeuroPET-Demo_raw/source/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._source  
   creating: OpenNeuroPET-Demo_raw/code/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._code  
  inflating: OpenNeuroPET-Demo_raw/README  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._README  
  inflating: OpenNeuroPET-Demo_raw/.bidsignore  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._.bidsignore  
  inflating: OpenNeuroPET-Demo_raw/dataset_description.json  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._dataset_description.json  
   creating: OpenNeuroPET-Demo_raw/sub-SiemensBiographNRU/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._sub-SiemensBiograph

In [11]:
!apt-get install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (264 kB/s)
Selecting previously unselected package tree.
(Reading database ... 124015 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [12]:
!tree OpenNeuroPET-Demo_raw

OpenNeuroPET-Demo_raw
├── code
│   ├── matlab_conversions.m
│   ├── python_conversions.sh
│   └── README.md
├── dataset_description.json
├── README
├── source
│   ├── SiemensBiographPETMR-NRU
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.100.2022.04.26.15.04.22.218.14690618.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.101.2022.04.26.15.04.22.218.14690629.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.10.2022.04.26.15.04.22.218.14689628.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.102.2022.04.26.15.04.22.218.14690640.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.103.2022.04.26.15.04.22.218.14690651.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.104.2022.04.26.15.04.22.218.14690662.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.105.2022.04.26.15.04.22.218.14690673.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.106.2022.04.26.15.04.22.218.14690684.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.107.2022.04.26.15.04.22.218.14690695.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_

In [16]:
!pip install dcm2niix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!dcm2niix4pet ./OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR-NRU -d mynewfolder

Attempting to locate missing BIDS fields in dicom header
FOUND BodyPartExamined corresponding to BIDS BodyPart: BRAIN
NOT FOUND MappingResourceName corresponding to BIDS TracerName in dicom header.
NOT FOUND CodeMeaning corresponding to BIDS TracerRadionuclide in dicom header.
NOT FOUND RadionuclideTotalDose corresponding to BIDS InjectedRadioactivity in dicom header.
NOT FOUND RadiopharmaceuticalSpecificActivity corresponding to BIDS MolarActivity in dicom header.
NOT FOUND RadiopharmaceuticalVolume corresponding to BIDS InjectedVolume in dicom header.
NOT FOUND InterventionDrugName corresponding to BIDS PharmaceuticalName in dicom header.
NOT FOUND InterventionDrugDose corresponding to BIDS PharmaceuticalDoseAmount in dicom header.
NOT FOUND RadiopharmaceuticalStartTime corresponding to BIDS InjectionStart in dicom header.
FOUND AcquisitionDate corresponding to BIDS ScanDate: 20220421
NOT FOUND RadiopharmaceuticalStopTime corresponding to BIDS InjectionEnd in dicom header.
FOUND Reco

In [18]:
!tree ./mynewfolder

./mynewfolder
├── Phantom_PetAcquisition_20.01.14-10_39_18-STD-1.3.12.2.1107.5.2.38.51014_20220421120447_30003.json
└── Phantom_PetAcquisition_20.01.14-10_39_18-STD-1.3.12.2.1107.5.2.38.51014_20220421120447_30003.nii.gz

0 directories, 2 files


In [19]:
!dcm2niix4pet ./OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR-NRU -d mynewfolder2 --kwargs TimeZero=ScanStart Manufacturer=Siemens ManufacturersModelName=Biograph InstitutionName="Rigshospitalet, NRU, DK" BodyPart=Phantom Units=Bq/mL TracerName=none TracerRadionuclide=F18 InjectedRadioactivity=81.24 SpecificRadioactivity=13019.23 ModeOfAdministration=infusion FrameTimesStart=0 AcquisitionMode="list mode" ImageDecayCorrected=true ImageDecayCorrectionTime=0 AttenuationCorrection=MR-corrected FrameDuration=300 FrameTimesStart=0

Attempting to locate missing BIDS fields in dicom header
FOUND BodyPartExamined corresponding to BIDS BodyPart: BRAIN
NOT FOUND MappingResourceName corresponding to BIDS TracerName in dicom header.
NOT FOUND CodeMeaning corresponding to BIDS TracerRadionuclide in dicom header.
NOT FOUND RadionuclideTotalDose corresponding to BIDS InjectedRadioactivity in dicom header.
NOT FOUND RadiopharmaceuticalSpecificActivity corresponding to BIDS MolarActivity in dicom header.
NOT FOUND RadiopharmaceuticalVolume corresponding to BIDS InjectedVolume in dicom header.
NOT FOUND InterventionDrugName corresponding to BIDS PharmaceuticalName in dicom header.
NOT FOUND InterventionDrugDose corresponding to BIDS PharmaceuticalDoseAmount in dicom header.
NOT FOUND RadiopharmaceuticalStartTime corresponding to BIDS InjectionStart in dicom header.
FOUND AcquisitionDate corresponding to BIDS ScanDate: 20220421
NOT FOUND RadiopharmaceuticalStopTime corresponding to BIDS InjectionEnd in dicom header.
FOUND Reco

## validate BIDS dataset
https://www.npmjs.com/package/bids-validator

In [ ]:
!pip install bids_validator

In [ ]:
from bids_validator import BIDSValidator

In [ ]:
BIDSValidator().is_bids('.') # not up to date, use browser version for now

## run only if freesurfer is installed

In [ ]:
!mkdir PETprocessing

In [ ]:
!cd PETprocessing

In [ ]:
!git clone https://github.com/openneuropet/PET_pipelines.git 

In [ ]:
!cd PET_pipelines/pyPetSurfer 

In [ ]:
!wget https://www.dropbox.com/sh/69dwtnv29wd7jlx/AADnw5FvAANpvzKAxVQTnyhBa?dl=0 

In [ ]:
!ls PET_pipelines/pyPetSurfer

In [ ]:
!unzip AADnw5FvAANpvzKAxVQTnyhBa?dl=0

In [ ]:
!pip install mri_coreg

In [ ]:
%run ./example.py

In [ ]:
!ls .
